In [ ]:
!pip install transformers==4.17.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 35.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('clean_data.csv')
df.head()

,film_name,gender,film_avg_rate,review_rate,review_title,review_text
0,Respira (Serie de TV),"Serie de TV, Drama, Medicina","4,6",1.0,Respira,"Como fan de las series españolas y de Najwa, e..."
1,Respira (Serie de TV),"Serie de TV, Drama, Medicina","4,6",2.0,Esta serie te llama tonto a la cara,Todo lo malo que puede tener una serie lo pose...
2,Respira (Serie de TV),"Serie de TV, Drama, Medicina","4,6",1.0,Respira,"La serie es un drama médico que intenta ""copia..."
3,Respira (Serie de TV),"Serie de TV, Drama, Medicina","4,6",1.0,"Imposible, no hay manera",Nadie te obliga a ver nada que no quieras ver ...
4,Respira (Serie de TV),"Serie de TV, Drama, Medicina","4,6",1.0,Serie nefasta,Está serie da vergüenza ajena. Una serie donde...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18659 entries, 0 to 18658
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   film_name      18659 non-null  object 
 1   gender         18659 non-null  object 
 2   film_avg_rate  18659 non-null  object 
 3   review_rate    18659 non-null  float64
 4   review_title   18659 non-null  object 
 5   review_text    18659 non-null  object 
dtypes: float64(1), object(5)
memory usage: 874.8+ KB


In [ ]:
from transformers import pipeline

In [ ]:
# Cargar modelo de análisis de sentimiento
clasificador = pipeline("text-classification", model="finiteautomata/beto-sentiment-analysis")

Downloading:   0%|          | 0.00/841 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/469k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:01<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Función para analizar el sentimiento de cada reseña
def obtener_sentimiento(texto):
    resultados = clasificador(texto, return_all_scores=True)
    positivo = next((res['score'] for res in resultados[0] if res['label'] == 'POS'), 0)
    negativo = next((res['score'] for res in resultados[0] if res['label'] == 'NEG'), 0)
    neutro = next((res['score'] for res in resultados[0] if res['label'] == 'NEU'), 0)
    return positivo, neutro, negativo

In [ ]:
def obtener_sentimiento_por_fragmentos(texto):
    # Dividir el texto en fragmentos de hasta 512 tokens
    max_length = 512
    palabras = texto.split()
    fragmentos = [" ".join(palabras[i:i+max_length]) for i in range(0, len(palabras), max_length)]

    # Analiza el sentimiento de cada fragmento
    resultados = [clasificador(fragmento, return_all_scores=True, truncation=True) for fragmento in fragmentos]

    # Promedia los resultados
    positivo = sum(next(res['score'] for res in resultado[0] if res['label'] == 'POS') for resultado in resultados) / len(resultados)
    negativo = sum(next(res['score'] for res in resultado[0] if res['label'] == 'NEG') for resultado in resultados) / len(resultados)
    neutro = sum(next(res['score'] for res in resultado[0] if res['label'] == 'NEU') for resultado in resultados) / len(resultados)

    return positivo, neutro, negativo


In [ ]:
# Aplica análisis de sentimientos
df[['positivo', 'neutro', 'negativo']] = df['review_text'].apply(lambda x: pd.Series(obtener_sentimiento_por_fragmentos(x)))

In [ ]:
# Verificar los resultados
print(df[['review_title', 'positivo', 'neutro', 'negativo']].head(10))

                                review_title  positivo    neutro  negativo
0                                    Respira  0.001416  0.000363  0.998221
1        Esta serie te llama tonto a la cara  0.000602  0.000522  0.998876
2                                    Respira  0.000626  0.000390  0.998985
3                   Imposible, no hay manera  0.000581  0.000187  0.999232
4                              Serie nefasta  0.000711  0.000434  0.998855
5                     Elitizando un hospital  0.007717  0.008993  0.983290
6         Crítica de Respira por Cinemagavia  0.016100  0.248398  0.735502
7              Médicos, relojes y actualidad  0.055379  0.043262  0.901359
8  Pequeña sorpresa en la sanidad valenciana  0.998135  0.000776  0.001089
9               No es tan mala como la ponen  0.008903  0.026627  0.964470


In [ ]:
df.to_csv('data_con_sentimientos.csv', index=False)